In [1]:
"""this python file is used to construct the fake data for the model"""
import random
import json
import numpy as np
import mysql.connector
import Tweet as tweet
from datetime import datetime

from networkx import *
import networkx.algorithms as nxalg

def read_data_from_database(script):
    mydb = mysql.connector.connect(
        user = "root",
        password = "D12345",
        host = "localhost",
        database = "tweetermysql_2018-07-30"
    )

    my_cursor = mydb.cursor()

    my_cursor.execute(script)

    my_result = my_cursor.fetchall()
    
    data = list()
    
    for item in my_result:
        u, m_data, date, section = item
        t = tweet.Tweet(u,section,m_data,datetime.strptime(date,'%Y-%m-%d'))
        data.append(t)
    
    return data

def create_word_id_dictionary(tweets):
    word_id_dict = {}
    id = 0
    for tweet in tweets:
        if not word_id_dict.get(tweet.user):
            word_id_dict[tweet.user] = id
            id += 1 
        if not word_id_dict.get(datetime.strftime(tweet.date,'%Y-%m-%d')):
            word_id_dict[datetime.strftime(tweet.date,'%Y-%m-%d')] = id
            monthFormat = "{month}.{year}".format(month=tweet.date.month,year=tweet.date.year)
            dayFormat = "{day}.{month}.{year}".format(day=tweet.date.day,month=tweet.date.month,year=tweet.date.year)
            word_id_dict[monthFormat] = id + 1
            word_id_dict[dayFormat] = id + 2
            id += 3
        for word in tweet.text.split():
            if not word_id_dict.get(word):
                word_id_dict[word] = id
                id += 1
    
    return word_id_dict
def find_max_node_size_for_tweet_text(tweets):
    max_node_size = len(tweets[0].text.split())
    #print(max_node_size, tweets[0].text.split())
    for tweet in tweets:
        if max_node_size < len(tweet.text.split()):
            max_node_size = len(tweet.text.split())
    return max_node_size
def create_time_graph(tweet):
    G1 = nx.DiGraph()
    #for tweet in tweets:
    #G1.add_node(word_id_dict[tweet.user])
    tweetFormat = str(tweet.Id)
    G1.add_node(str(tweet.date.year))
    #G1.add_edge(word_id_dict[tweet.user],tweetFormat,weight=1.0)
    #G1.add_edge(YEAR_TAG,tweet.date.year,weight=15.0)
    #G.add_edge(tweet.date.year,YEAR_TAG,weight=1.0)
    for word in tweet.text.split():
        G1.add_edge(word_id_dict[word],tweetFormat,weight=1.0)
        
    monthFormat = "{month}.{year}".format(month=tweet.date.month,year=tweet.date.year)

    G1.add_edge(str(tweet.date.year),word_id_dict[monthFormat],weight=1.0)
    G1.add_edge(word_id_dict[monthFormat],str(tweet.date.year),weight=1.0)

    dayFormat = "{day}.{month}.{year}".format(day=tweet.date.day,month=tweet.date.month,year=tweet.date.year)
    G1.add_edge(word_id_dict[monthFormat],word_id_dict[dayFormat],weight=1.0)
    G1.add_edge(word_id_dict[dayFormat],tweetFormat,weight=1.0)
    return G1

def create_random_graph(tweets, filePath, graph_scale):
    """

    :param type: the graph type
    :param filePath: the output file path
    :param numberOfCase: the number of examples
    :return:
    """
    with open(filePath, "w+", encoding='utf-8') as f:
        degree = 0.0
        for i in range(len(tweets)):
            info = {}
            graph_node_size = graph_scale
            graph_real_node_size = graph_scale
            edge_prob = 0.3

            edge_count = 0.0

            graph = create_time_graph(tweets[i])
            #graph = nx.convert_node_labels_to_integers(graph,first_label=0)
            #graph = nx.DiGraph(),
            k = len(graph.nodes)
            graph_real_node_size = k
            ancestor = ""
            if k < graph_scale:
                while k <= graph_scale:
                    graph.add_node(str(k))
                    k+=1

            for id in graph.edges:
                edge_count += len(graph.edges[id])
            graph_node_size = len(graph.nodes)
            start = 0
            #nodes = list(graph.nodes)
            end = len(graph.nodes)-1
            #path = nx.shortest_path(graph, nodes[0], nodes[len(graph.nodes)-1])
            #paths = [p for p in nx.all_shortest_paths(graph, nodes[0], nodes[len(graph.nodes)-1])]
            #if len(path) >= 3 and len(paths) == 1:
            #    degree += edge_count / len(graph.nodes)
            #    break


            adj_list = []
            for adj in graph.adjacency():
                adj_list.append(list(adj[1].keys()))
            g_nodes = []
            for node in graph.nodes:
                g_nodes.append(node)
            #print(adj_list[0])

            g_ids = {}
            g_ids_features = {}
            g_adj = {}
            #print(graph_node_size,start,end)
            for j in range(graph_node_size):
                g_ids[j] = int(g_nodes[j])
                if j == start:
                    g_ids_features[j] = "START"
                elif j == end:
                    g_ids_features[j] = "END"
                else:
                    #g_ids_features[i] = str(i+10)
                    g_ids_features[j] ="1"
                g_adj[j] = adj_list[j]

            # print start, end, path
            text = "START "+ tweets[i].user + " END "
            #for id in range(len(path)):
            #    text += g_ids_features[nodes[id]] + " "

            info["seq"] = text.strip()
            info["g_ids"] = g_ids
            info['g_ids_features'] = g_ids_features
            info['g_adj'] = g_adj
            f.write((json.dumps(info,ensure_ascii=False)+"\n"))



In [2]:
if __name__ == "__main__":
    script = 'select ID, MetaData, Date,Section from newsdunya_all_sektorler where Section = "Otomotiv" or Section ="Enerji" or Section = "Emlak" limit 300'
    tweets = read_data_from_database(script)
    graph_scale = find_max_node_size_for_tweet_text(tweets) + 5 
    word_id_dict = create_word_id_dictionary(tweets)
    #print(word_id_dict)
    create_random_graph(tweets, "./Graph2Seq/data/no_cycle/train.data", graph_scale)
    script =  ' select ID, MetaData, Date,Section from newsdunya_all_sektorler where Section = "Otomotiv" or Section ="Enerji" or Section = "Emlak" limit 300,300'
    tweets = read_data_from_database(script)
    word_id_dict = create_word_id_dictionary(tweets)
    create_random_graph(tweets, "./Graph2Seq/data/no_cycle/dev.data", graph_scale)
    create_random_graph(tweets, "./Graph2Seq/data/no_cycle/test.data", graph_scale)
